In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [5]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol, shift_symbols_down
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
translator = CirqTranslater(3, untouchable_blocks = [1])
translator.env_qubits = [2]

translator.encoder_id = 0
translator.channel_id = 1
translator.decoder_id = 2

### state prep

u2_layer_encoder = u2_db(translator, 0,1, block_id=translator.encoder_id)
channel_db = amplitude_damping_db(translator, qubits_ind=[1,2], eta=1, block_id = translator.channel_id)
u2_layer_decoder = u2_db(translator, 0,1, block_id=translator.decoder_id)

circuit_db = concatenate_dbs([u2_layer_encoder, channel_db, u2_layer_decoder])

circuit, circuit_db = translator.give_circuit(circuit_db)

#circuit

minimizer = Minimizer(translator, mode="discrimination")
etas = [0.01, 1.]


batch_circuits, trainable_symbols, trainable_params_value = prepare_optimization_discrimination(translator, circuit_db, etas)
cost, resolver, training_history = minimizer.minimize(batch_circuits, symbols = trainable_symbols, parameter_values = trainable_params_value )

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().


In [3]:
circuit_db = translator.update_circuit_db_param_values(circuit_db, resolver)
simplifier = Simplifier(translator)
simplified_db, nreds = simplifier.reduce_circuit(circuit_db)

In [4]:
simplified_batch_circuits, _ = channel_circuits(translator, simplified_db, etas )
minimizer.give_cost(simplified_batch_circuits)

<tf.Tensor: shape=(), dtype=float32, numpy=0.14601386>

In [6]:
inserter = IdInserter(translator.n_qubits, untouchable_blocks=translator.channel_id)

In [7]:
mutated_circuit_db = inserter.insert_many_mutations(circuit_db )

In [9]:
batch_circuits, trainable_symbols, trainable_params_value = prepare_optimization_discrimination(translator, mutated_circuit_db, etas)
cost, resolver, training_history = minimizer.minimize(batch_circuits, symbols = trainable_symbols, parameter_values = trainable_params_value )

In [60]:
minimizer.give_cost(channel_circuits(translator, circuit_db, etas )[0], resolver = translator.give_resolver(circuit_db),model = minimizer.model)

<tf.Tensor: shape=(), dtype=float32, numpy=0.4793837>

In [59]:
minimizer.give_cost(channel_circuits(translator, simplified_db, etas )[0], resolver = translator.give_resolver(circuit_db), model = minimizer.model)

<tf.Tensor: shape=(), dtype=float32, numpy=0.4793837>

In [58]:
minimizer.loss(*[minimizer.model(batch_circuits)]*2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.14601326>

In [62]:
minimizer.model(batch_circuits)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 0.99979985, -0.9999999 ,  0.99979985],
       [-0.41614735, -0.9999999 , -0.41614735]], dtype=float32)>

In [63]:
sbatch_circuits, batch_circuits_db = channel_circuits(translator, simplified_db, etas )